# Discarded Seafloor-objects: A Hidden World Beneath the Waves

#### A Project by Allison Buchanan and Robyn Marowitz
![Radioactive Dumping](img/environmental-destruction-waste-plastic-plastic-waste.jpeg)
image credit: pxfuel.com

## Introduction to the Sea Floor

The ocean's depths are a realm of mystery and discovery, holding wonders that have intrigued scientists and explorers for centuries.  
Sadly, the marine environment halso as been a dumping ground for discarded objects for decades. An astonishing variety of discarded objects rest quietly beneath the waves. These forgotten artifacts range from ancient shipwrecks to today's waste; from gigantic swaths of fishing nets to modern-day debris. Some of these objects are pretty hazardous. One example is an estimated 55,000 containers of radioactive waste. These containers were dumped overboard at various Pacific Ocean sites from 1946-1970 (epa., 2022gov).  

New technology allows us to investigate these objects and determine their outcomes. We can employ machine learning to determine what might happen to titemsjects- will they sink, be buried, or drift? And, if they drift, where will they go? These methods help us determine where hotspots of discarded objects are and help us make more informed decisions and advances in our environmental, ecological and navigational movements. 


![Ocean Dumping](img/seabed-pollution.jpeg)

image credit: Public Domain Pictures, 2022

In [1]:
import os
import glob
import earthpy as et
import earthpy.plot as ep
import pandas as pd
import geopandas as gpd
import seaborn as sns
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
from shapely.geometry import Polygon
from fiona.drvsupport import supported_drivers
import folium
from folium.plugins import MarkerCluster
import xarray as xr
import rioxarray as rxr
import rasterio as rio
from rasterio.plot import plotting_extent
import zipfile

# python file containing our functions
from kml_functions import read_kml, create_refined_df
from tif_functions import read_tifs, night_lights_tif_to_df, open_tif_to_df

# go to /earth-analytics/data directory
os.chdir(os.path.join(et.io.HOME, 'earth-analytics', 'data'))

# Create Bounding Box - North and South Carolina coastal region
box = {'geometry': [Polygon([(-77.121369, 36.541466),
                             (-70.760165, 36.541466),
                             (-71.511922, 32.087495),
                             (-79.317663, 31.036502)])]}
bbox_gdf = gpd.GeoDataFrame(box, crs='EPSG:4326')
bbox_gdf.bounds

# NOTE: Download instructions are in the README for this.
nc_reef_kml_pth = os.path.join(
   et.io.HOME, 'earth-analytics', 'data', 'earthpy-downloads',
   'Reef_Material.kml')

sc_reef_kml_pth = os.path.join(
   et.io.HOME, 'earth-analytics', 'data', 'earthpy-downloads',
   'ArtReef2021__GE', 'ArtReef2021.kml')

if not os.path.exists(sc_reef_kml_pth):
    et.data.get_data(
        url=('https://www.dnr.sc.gov/marine/reef/ArtReef2021__GE.zip'))

# Get datasets from the Wrecks and Obstructions Database
ENC_wrecks_kml_pth = os.path.join(
    et.io.HOME, 'earth-analytics', 'data', 'earthpy-downloads',
    'ENC_Wrecks.kml')
AWOIS_wrecks_kml_pth = os.path.join(
    et.io.HOME, 'earth-analytics', 'data', 'earthpy-downloads',
    'AWOIS_Obstructions.kml')

# Caching
if not os.path.exists(ENC_wrecks_kml_pth):
    et.data.get_data(
        url=("https://wrecks.nauticalcharts.noaa.gov/"
             "downloads/ENC_Wrecks.kml"))
if not os.path.exists(AWOIS_wrecks_kml_pth):
    et.data.get_data(
        url="https://wrecks.nauticalcharts.noaa.gov/"
            "downloads/AWOIS_Obstructions.kml")

# ENC Wrecks gdf
ENC_gdf = read_kml(ENC_wrecks_kml_pth)
ENC_df = create_refined_df(ENC_gdf)
ENC_df['description'] = 'ENC Wreck'
ENC_df.reset_index()

# AWOIS Wrecks/obstrctions
AWOIS_gdf = read_kml(AWOIS_wrecks_kml_pth)
AWOIS_df = create_refined_df(AWOIS_gdf)
AWOIS_df['description'] = 'AWOIS Obstruction'
AWOIS_df

# North Carolina reef material/oyster sanctuaries 
#nc_reef_gdf = read_kml(nc_reef_kml_pth)
#type(nc_reef_gdf)
#nc_reef_df = pd.DataFrame(columns=[
 #   'lat', 'lon', 'size', 'description'])
#nc_reef_df['lon'] = nc_reef_gdf.geometry.centroid.x
#nc_reef_df['lat'] = nc_reef_gdf.geometry.centroid.y
#nc_reef_df['description'] = nc_reef_gdf.Name

# SC Reefs
sc_reef_gdf = read_kml(sc_reef_kml_pth)
sc_reef_df = create_refined_df(sc_reef_gdf)
sc_reef_df['description'] = sc_reef_gdf.Name

## The Process

Our chosen study site was an area off the coast of the Carolinas. We collect d ancombineded a wide range of data- shipwrecks, artificial reefs, curren and tswaes patterns, boating traffic, and more- to see if we could identify the fate of located objes.ct On o interactiveur map othe Carolina coast shownea bel,othere isee point data of different materials that we are investigating during this proces As the map is zoomed in more detailed information becomes available about the identified objects. This data is what we can use to see how our inferential data such as currents and sediment are impacting physical objects.



In [2]:
total_df = pd.concat([sc_reef_df, AWOIS_df, ENC_df])
# (Threw out nc_reef_df from this for the sake of getting it to run. 
# nc_reef_df,

# Map our concatonated dataframe
m = folium.Map(location=[32.087495, -71.511922], zoom_start=6)

marker_cluster = MarkerCluster().add_to(m)

for index, row in total_df.iterrows():
    folium.Marker(
        location=[row.lat, row.lon],
        popup=row.description,
        icon=folium.Icon(color="black")
    ).add_to(marker_cluster)

m

### NearestNeighbor

Once our data was conformed, we created two different joined sets of data to perform machine learning on. One set of data trains the model, the other set is what we look for correlation in. These are known as the "training data" and "testing data." We chose the NearestNeighbor Analysis from ScikitLearn for our machine learning model because it measures the spread or distribution of something over a geographical space. We also made sure to choose an algorithm that can handle large sample sizes.

## Analysis & Findings

As of the date of this assignment, our analysis is still not complete. Our progress has been wrought with problems all along. To begin with, we have an overwhelming amount of data to bring into our assignment. This alone took a lot of work. Then we had to join all these various sorts and types of data together into cohesive data frames. Then we had hardware issues. The Windows computer we are using could not solve our environment and so we had to build a docker container. This meant we had to learn the in's and out's of troubleshooting that platform which added a layer of complexity to our project. Docker is kind of like a separate virtual environment without all the resources and time that a true virtual machine may take, so it allows us to work from an environment without needing the environment on the computer itself. Once we got docker up and running successfully we then realized that the docker container for our project is taking up a huge amount of RAM and now that we have begun to join larger arrays that lack of dynamic memory is becoming a problem. This is a new problem that we are currently trying to work around while also try to perform the machine learning aspect of our project. 


## Final Thoughts

Our data was.... 

The ocean is a massively important place for us as a planet. Due to its size and depth, tracking what happens on the seafloor has traditionally been very hard. Having cutting-edge methods such as the one we employed in this study will help us to develop environmental models that can aid in sustainable fishing practices, enhance our navigational safety, help us understand climate change impacts, and so much more. Just last year a report came out that stated that a glacier machine learning model- the Instructed Glacier Model- was developed to predict the evolution of glaciers and ice sheets up to 1000 times faster than any previous methods (Columbia Climate School, 2022). This extreme increase in speed is due to the fact that the heavy physics that is involved in modeling is handled by AI and therefore can be processed much faster. This model will greatly enhance our ability to predict sea level rise and is only one of many examples of how this type of technology may positively impact our society. 

## Want to learn more? 

Please visit our git hub repository for more technical information and to view our software: https://github.com/rmarowitz/seafloor-objects
You may also contact us at allisonwiddecombe@gmail.com or Robyn.Marowitz@colorado.edu

## Thank you for your interest! 

## Citations

"Learn about Ocean Dumping," epa.gov. October 27, 2022.
"Machine Learning Techniques Can Speed Up Glacier Modeling By A Thousand Times," Glacierhub Blog, Columbia Climate School. March 25, 2022.